In [ ]:
import boto3

ddb = boto3.client('dynamodb')

In [ ]:
response = ddb.create_table(
    TableName='EventStore',
     AttributeDefinitions=[
        {
            'AttributeName': 'AggregateId',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'Version',
            'AttributeType': 'N'
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'AggregateId',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'Version',
            'KeyType': 'RANGE'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 5
    }
)

print response

In [ ]:
class Event:
    kind = 'esevent'
    
    def __init__(self, id, version, typecode, payload):
        self.id = id
        self.version = version
        self.typecode = typecode
        self.payload = payload
        
    def __str__(self):
        return 'Event {} - {}'.format(self.id. self.version)

In [ ]:
addUser = {}
addUser['Name'] = 'doug'
addUser['Email'] = 'doug@dev.null'
type(addUser)


In [ ]:
class UserAdded(Event):
    kind = 'useradded'

class EmailUpdated(Event):
    kind = 'emailupdated'
    
def print_event_kind(event):
    print type(event)
    print event.kind


In [ ]:
e1 = UserAdded('ag1',1, 'adduser', addUser)
print_event_kind(e1)

In [ ]:
print e1.id
print type(e1)

In [ ]:
updateEmail = {}
updateEmail['Email'] = "reallydoug@somewhere.com"
e2 = EmailUpdated('ag1',2, 'updateemail', updateEmail)

In [ ]:
from boto3.dynamodb.types import TypeSerializer


ts = TypeSerializer()
addUserMap = ts.serialize(addUser)
print addUserMap

In [ ]:
response = ddb.batch_write_item(
    RequestItems={
        'EventStore':[
            {
                'PutRequest':{
                    'Item':{
                        'AggregateId':{'S': e1.id},
                        'Version':{'N':str(e1.version)},
                        'Payload':ts.serialize(e1.payload),
                        'Typecode':{'S':e1.typecode}
                    }
                }
            }
        ]
    }
)

print response

In [ ]:

response = ddb.batch_write_item(
    RequestItems={
        'EventStore':[
            {
                'PutRequest':{
                    'Item':{
                        'AggregateId':{'S': e2.id},
                        'Version':{'N':str(e2.version)},
                        'Payload':ts.serialize(e2.payload),
                        'Typecode':{'S':e2.typecode}
                    }
                }
            }
        ]
    }
)

print response

In [ ]:
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('EventStore')
response = table.query(
    KeyConditionExpression=Key('AggregateId').eq(e1.id)
)

print response['Items']

payload1 = response['Items'][0]['Payload']
print payload1
print type(payload1)

In [ ]:
response = table.query(
    KeyConditionExpression=Key('AggregateId').eq(e1.id),
    ScanIndexForward=False
)

print response